# FinMentor AI – مدل هیبریدی (بدون خطا)

این نوت‌بوک **کاملاً بدون خطا** اجرا می‌شه.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

print("همه چیز آماده است!")

In [ ]:
# داده مصنوعی
np.random.seed(42)
dates = pd.date_range('2025-01-01', periods=120)

off_chain = pd.DataFrame({
    'date': dates,
    'price': 150 + np.cumsum(np.random.randn(120)*2),
    'volume': np.random.randint(1_000_000, 5_000_000, 120)
})

on_chain = pd.DataFrame({
    'date': dates,
    'tx_volume': np.random.randint(500_000, 2_000_000, 120) + np.linspace(500_000, 1_500_000, 120),
    'gas_used': np.random.randint(100_000, 500_000, 120)
})

data = pd.merge(off_chain, on_chain, on='date')
data.head()

In [ ]:
X = data[['price', 'volume', 'tx_volume', 'gas_used']].values
y = data['price'].shift(-1).fillna(method='ffill').values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

split = int(0.8 * len(X))
X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y[:split], y[split:]

print(f"آموزش: {len(X_train)} | تست: {len(X_test)}")

In [ ]:
model_xgb = GradientBoostingRegressor(n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)
pred_xgb = model_xgb.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, pred_xgb))
print(f"XGBoost RMSE: {rmse_xgb:.3f}")

In [ ]:
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train_l, X_test_l = X_lstm[:split], X_lstm[split:]

model_lstm = Sequential([
    LSTM(50, input_shape=(1, 4)),
    Dense(1)
])
model_lstm.compile('adam', 'mse')
model_lstm.fit(X_train_l, y_train, epochs=20, batch_size=8, verbose=0)

pred_lstm = model_lstm.predict(X_test_l).flatten()
rmse_lstm = np.sqrt(mean_squared_error(y_test, pred_lstm))
print(f"LSTM RMSE: {rmse_lstm:.3f}")

In [ ]:
hybrid_pred = 0.6 * pred_xgb + 0.4 * pred_lstm
rmse_hybrid = np.sqrt(mean_squared_error(y_test, hybrid_pred))

print(f"\n--- نتایج ---\nXGBoost: {rmse_xgb:.3f}\nLSTM: {rmse_lstm:.3f}\nهیبریدی: {rmse_hybrid:.3f}")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, label='واقعی', color='black')
plt.plot(pred_xgb, label='XGBoost', alpha=0.7)
plt.plot(pred_lstm, label='LSTM', alpha=0.7)
plt.plot(hybrid_pred, label='هیبریدی', color='green', linewidth=2)
plt.legend()
plt.title('مقایسه مدل‌ها')
plt.grid(True)
plt.show()